## 資料集拆分

In [2]:
import os
import shutil
import numpy as np
import scipy.io
from collections import Counter
from tqdm import tqdm
import re

# ----------------------------------------------------------------
# 路徑設定
# ----------------------------------------------------------------
# .mat 標記檔的路徑
mat_path = r"D:\deeplearning_class\dl_hw3\mpii_human_pose_v1_u12_1.mat"
# 原始圖片資料夾
images_dir = r"D:\deeplearning_class\dl_hw3\images"

# 輸出根目錄：將所有分類都放在這個資料夾底下，每個子資料夾就是一個 label
output_base = r"D:\deeplearning_class\dl_hw3\mpii_image_classification_single_folder"
# ----------------------------------------------------------------

# 讀取 .mat 檔案
mat = scipy.io.loadmat(mat_path, struct_as_record=False, squeeze_me=True)
release = mat['RELEASE']

annolist = release.annolist       # 圖片列表結構
# train/test 標記（不會用到，但先讀取以保留原本流程）
img_train_flags = release.img_train
# 動作標籤結構（如果有 act 欄位）
acts = release.act if 'act' in release._fieldnames else None


# 定義：清理非法字元的函式，回傳可作為資料夾名稱的字串
def sanitize_folder_name(name):
    return re.sub(r'[<>:"/\\|?*]', '', name).strip()


# 定義：根據索引 idx 讀取動作名稱，回傳「文字」
def get_action_label(idx):
    if acts is None or len(acts) <= idx:
        return "unknown"
    act_struct = acts[idx]

    # 如果有 act_name 或 cat_name，取其一
    label = None
    if hasattr(act_struct, 'act_name'):
        label = act_struct.act_name
    elif hasattr(act_struct, 'cat_name'):
        label = act_struct.cat_name

    # 如果 label 是陣列或 list，就取第一個元素
    if isinstance(label, (np.ndarray, list)):
        if len(label) > 0:
            label = label[0]
        else:
            return "unknown"
    # 如果是 bytes，就 decode
    if isinstance(label, bytes):
        label = label.decode('utf-8')
    # 如果是空字串或 None，就視為 unknown
    if label is None or label == "":
        return "unknown"

    return sanitize_folder_name(label)


# ----------------------------------------------------------------
# 1. 先算出每張圖片的 label，並統計各 label 的圖片數量
# ----------------------------------------------------------------
all_labels = [get_action_label(i) for i in range(len(annolist))]
label_counts = Counter(all_labels)

# 過濾出「圖片數量 ≥ 200 張」的 label
valid_labels = {label for label, count in label_counts.items() if count >= 200}

print(f"總類別數: {len(label_counts)}")
print(f"過濾後類別數（圖片 ≥ 200 張）: {len(valid_labels)}")

# 如果你想要先把所有 valid_labels 的資料夾通通預先建立好，可以用下面這段
for label in valid_labels:
    label_dir = os.path.join(output_base, label)
    os.makedirs(label_dir, exist_ok=True)


# ----------------------------------------------------------------
# 2. 把所有滿足條件（label ∈ valid_labels）的圖片都拷貝到 output_base/label 下
# ----------------------------------------------------------------
for idx in tqdm(range(len(annolist)), desc="Copying images"):
    label = get_action_label(idx)
    # 只處理在 valid_labels 裡面的 label
    if label not in valid_labels:
        continue

    # 取得圖片檔名
    ann = annolist[idx]
    img_name = ann.image.name  # e.g. "00000001.jpg"

    src_img_path = os.path.join(images_dir, img_name)
    dst_dir = os.path.join(output_base, label)
    dst_img_path = os.path.join(dst_dir, img_name)

    # 如果來源圖片不存在，就跳過並報告警告
    if not os.path.exists(src_img_path):
        print(f"警告：圖片不存在，跳過 -> {src_img_path}")
        continue

    # 複製圖片到對應 label 資料夾
    shutil.copy2(src_img_path, dst_img_path)

print("所有分類整理完成！")


總類別數: 398
過濾後類別數（圖片 ≥ 200 張）: 5


Copying images: 100%|██████████| 24987/24987 [00:25<00:00, 970.16it/s] 

所有分類整理完成！


In [ ]:
#切割train val test
import os
import shutil
import random
from pathlib import Path

def split_dataset(input_dir, output_dir, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, seed=42):
    random.seed(seed)
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)

    # 確保比例加起來是1.0
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratio must sum to 1.0"

    # 建立資料夾結構
    for split in ['train', 'val', 'test']:
        (output_dir / split).mkdir(parents=True, exist_ok=True)

    # 對每一個類別資料夾進行分割
    for class_dir in input_dir.iterdir():
        if not class_dir.is_dir():
            continue
        images = list(class_dir.glob("*"))
        random.shuffle(images)

        n = len(images)
        n_train = int(train_ratio * n)
        n_val = int(val_ratio * n)
        n_test = n - n_train - n_val

        split_data = {
            'train': images[:n_train],
            'val': images[n_train:n_train+n_val],
            'test': images[n_train+n_val:]
        }

        for split, split_images in split_data.items():
            class_output_dir = output_dir / split / class_dir.name
            class_output_dir.mkdir(parents=True, exist_ok=True)

            for img_path in split_images:
                shutil.copy(img_path, class_output_dir / img_path.name)

    print("✅ Dataset split complete!")

if __name__ == "__main__":
    # 修改為你的實際路徑
    input_folder = output_base
    
    #!!!!!!!!修改這個!!!!!!!結果資料在這
    output_folder = "./mpii_image_classification_200"

    split_dataset(input_folder, output_folder)


✅ Dataset split complete!


In [4]:
#印出資料集個數
import os
from collections import Counter

# 設定資料集根目錄
root_dir = output_folder

# 要處理的三個子資料夾
splits = ['train', 'val', 'test']

# 用來儲存最終統計結果
# 格式：{ 'train': {'bicycling, mountain': 123, 'cooking or food preparation': 456, ...}, 
#         'val': {...}, 
#         'test': {...} }
split_counts = {}

for split in splits:
    split_path = os.path.join(root_dir, split)
    # 如果這個 split 資料夾不存在，就跳過
    if not os.path.isdir(split_path):
        print(f"Warning: 找不到資料夾 {split_path}，已跳過。")
        continue
    
    # 先取得該 split 底下所有子資料夾名稱，也就是各個 class
    class_names = [d for d in os.listdir(split_path) 
                   if os.path.isdir(os.path.join(split_path, d))]
    
    # 用 Counter 來計算每個 class 底下有多少檔案
    cnt = Counter()
    for cls in class_names:
        cls_folder = os.path.join(split_path, cls)
        
        # 把該 class 資料夾裡所有檔案列出來
        # 這裡我們只簡單地算所有非資料夾的檔案數量
        # 如果你想只算 .jpg、.png，可以再加檔名篩選
        file_list = [
            f for f in os.listdir(cls_folder) 
            if os.path.isfile(os.path.join(cls_folder, f))
            # 範例：只統計 .jpg/.jpeg/.png，如果沒差就把下面那行拿掉
            and f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]
        cnt[cls] = len(file_list)
    
    split_counts[split] = cnt

# 印出各 split 底下每個 class 的統計
for split, counter_obj in split_counts.items():
    print(f"--- {split.upper()} 集合 ---")
    # 依照類別名稱排序輸出 (可改成依照數量排序：counter_obj.most_common())
    for cls_name in sorted(counter_obj.keys()):
        print(f"  類別 '{cls_name}': {counter_obj[cls_name]} 張")
    print()
    
# 如果需要把三個 split 的各 class 數量累加（aggregate）成一張表，也可以這樣做：
agg_counter = Counter()
for split, counter_obj in split_counts.items():
    for cls_name, cnt in counter_obj.items():
        agg_counter[cls_name] += cnt

print("--- 全部 (train + val + test) 累計統計 ---")
for cls_name in sorted(agg_counter.keys()):
    print(f"  類別 '{cls_name}': {agg_counter[cls_name]} 張")


--- TRAIN 集合 ---
  類別 'bicycling, mountain': 172 張
  類別 'cooking or food preparation': 166 張
  類別 'skiing, downhill': 170 張
  類別 'unknown': 5563 張
  類別 'yoga, Power': 229 張

--- VAL 集合 ---
  類別 'bicycling, mountain': 21 張
  類別 'cooking or food preparation': 20 張
  類別 'skiing, downhill': 21 張
  類別 'unknown': 695 張
  類別 'yoga, Power': 28 張

--- TEST 集合 ---
  類別 'bicycling, mountain': 22 張
  類別 'cooking or food preparation': 22 張
  類別 'skiing, downhill': 22 張
  類別 'unknown': 696 張
  類別 'yoga, Power': 30 張

--- 全部 (train + val + test) 累計統計 ---
  類別 'bicycling, mountain': 215 張
  類別 'cooking or food preparation': 208 張
  類別 'skiing, downhill': 213 張
  類別 'unknown': 6954 張
  類別 'yoga, Power': 287 張
